In [1]:
from monte_carlo_ar1 import generate_staggered_law_ar1_data

In [2]:
data = generate_staggered_law_ar1_data(50,20,0.5,10)

In [8]:
data.head()

,state,individual,time,value,state_2,state_3,state_4,state_5,state_6,state_7,...,time_11,time_12,time_13,time_14,time_15,time_16,time_17,time_18,time_19,TREATMENT
0,1,1,0,-1.287607,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,0,1.096151,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,3,0,1.318206,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,4,0,-1.386582,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,0,1.156923,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data.columns

Index(['state', 'individual', 'time', 'value', 'state_2', 'state_3', 'state_4',
       'state_5', 'state_6', 'state_7', 'state_8', 'state_9', 'state_10',
       'state_11', 'state_12', 'state_13', 'state_14', 'state_15', 'state_16',
       'state_17', 'state_18', 'state_19', 'state_20', 'state_21', 'state_22',
       'state_23', 'state_24', 'state_25', 'state_26', 'state_27', 'state_28',
       'state_29', 'state_30', 'state_31', 'state_32', 'state_33', 'state_34',
       'state_35', 'state_36', 'state_37', 'state_38', 'state_39', 'state_40',
       'state_41', 'state_42', 'state_43', 'state_44', 'state_45', 'state_46',
       'state_47', 'state_48', 'state_49', 'state_50', 'time_1', 'time_2',
       'time_3', 'time_4', 'time_5', 'time_6', 'time_7', 'time_8', 'time_9',
       'time_10', 'time_11', 'time_12', 'time_13', 'time_14', 'time_15',
       'time_16', 'time_17', 'time_18', 'time_19', 'TREATMENT'],
      dtype='object')

In [11]:
df = data.copy()

In [12]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import GMM

# Assuming 'df' is your DataFrame

class TwoEquationGMM(GMM):
    def __init__(self, endog, exog, instrument1, instrument2):
        # Combine both sets of instruments
        instruments = np.column_stack((instrument1, instrument2))
        super(TwoEquationGMM, self).__init__(endog, exog, instruments=instruments)

    def momcond(self, params):
        # Unpack parameters
        beta, delta = params[:-1], params[-1]

        # Moment conditions for the first equation
        moment1 = (self.endog[self.instrument[:, -1] == 0] - 
                   self.exog[self.instrument[:, -1] == 0] @ beta)[:, None] * self.instrument[self.instrument[:, -1] == 0, :-1]

        # Moment conditions for the second equation
        moment2 = (self.endog - self.exog @ beta - self.exog[:, -1] * delta)[:, None] * self.instrument[:, -1][:, None]

        return np.concatenate([moment1, moment2], axis=0)

# Extracting the state and time dummies
state_time_dummies = df.filter(regex='^state_|^time_')

# Prepare the data
endog = df['value']
exog = sm.add_constant(df[['TREATMENT']])  # Include other covariates as necessary
treatment_indicator = df['TREATMENT'].values

# Initialize and fit the model
initial_params = np.zeros(exog.shape[1] + 1)  # Adjust initial parameter guesses as needed
model = TwoEquationGMM(endog, exog, state_time_dummies.values, treatment_indicator[:, None])
result = model.fit(start_params=initial_params)

# Display results
print(result.summary())


TypeError: GMM.__init__() missing 1 required positional argument: 'instrument'

In [14]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import GMM

# Assuming 'df' is your DataFrame

class TwoEquationGMM(GMM):
    def __init__(self, endog, exog, instrument):
        super(TwoEquationGMM, self).__init__(endog, exog, instrument=instrument)

    def momcond(self, params):
        # Unpack parameters
        beta, delta = params[:-1], params[-1]

        # Moment conditions for the first equation
        is_treatment_zero = (self.instrument[:, -1] == 0)
        moment1 = (self.endog[is_treatment_zero] - 
                   self.exog[is_treatment_zero] @ beta)[:, None] *  self.instrument[is_treatment_zero, :-1]

        # Moment conditions for the second equation
        moment2 = (self.endog - self.exog @ beta - self.exog[:, -1] * delta)[:, None] * self.instrument[:, -1][:, None]

        return np.concatenate([moment1, moment2], axis=0)

# Extracting the state and time dummies
state_time_dummies = df.filter(regex='^state_|^time_')

# Combine state_time_dummies with the treatment indicator for the first equation
instrument1 = np.hstack([state_time_dummies, (df['TREATMENT'] == 0)[:, None]])

# Treatment indicator for the second equation
instrument2 = df['TREATMENT'][:, None]

# Combine the instruments for both equations
instruments = np.hstack([instrument1, instrument2])

# Prepare the data
endog = df['value']
exog = sm.add_constant(df[['TREATMENT']])  # Include other covariates as necessary

# Initialize and fit the model
initial_params = np.zeros(exog.shape[1] + 1)  # Adjust initial parameter guesses as needed
model = TwoEquationGMM(endog, exog, instruments)
result = model.fit(start_params=initial_params)

# Display results
print(result.summary())


C:\Users\sneha\AppData\Local\Temp\ipykernel_19460\678591682.py:30: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  instrument1 = np.hstack([state_time_dummies, (df['TREATMENT'] == 0)[:, None]])
C:\Users\sneha\AppData\Local\Temp\ipykernel_19460\678591682.py:33: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  instrument2 = df['TREATMENT'][:, None]


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 69 and the array at index 1 has size 1

In [16]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import GMM

# Assuming 'df' is your DataFrame

class TwoEquationGMM(GMM):
    def __init__(self, endog, exog, instrument1, instrument2):
        # Combining the instruments for both equations
        self.instrument1 = instrument1
        self.instrument2 = instrument2
        # The total instrument matrix is just a placeholder in this case
        instruments = np.column_stack((instrument1, instrument2))  
        super(TwoEquationGMM, self).__init__(endog, exog, instrument=instruments)

    def momcond(self, params):
        # Unpack parameters
        beta, delta = params[:-1], params[-1]

        # Moment conditions for the first equation
        moment1 = (self.endog[self.instrument1[:, 0] == 0] - 
                   self.exog[self.instrument1[:, 0] == 0] @ beta)[:, None] * self.instrument1[self.instrument1[:, 0] == 0, 1:]

        # Moment conditions for the second equation
        moment2 = (self.endog - self.exog @ beta - self.exog[:, -1] * delta)[:, None] * self.instrument2[:, None]

        return np.concatenate([moment1, moment2], axis=0)

# Extracting the state and time dummies
state_time_dummies = df.filter(regex='^state_|^time_')

# Instrument for the first equation (state and time dummies)
instrument1 = np.hstack([state_time_dummies, (df['TREATMENT'] == 0)[:, None]])

# Instrument for the second equation (treatment indicator)
instrument2 = df['TREATMENT'].values

# Prepare the data
endog = df['value']
exog = sm.add_constant(df[['TREATMENT']])  # Include other covariates as necessary

# Initialize and fit the model
initial_params = np.zeros(exog.shape[1] + 1)  # Adjust initial parameter guesses as needed
model = TwoEquationGMM(endog, exog, instrument1, instrument2)
result = model.fit(start_params=initial_params)

# Display results
print(result.summary())


C:\Users\sneha\AppData\Local\Temp\ipykernel_19460\2546762321.py:34: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  instrument1 = np.hstack([state_time_dummies, (df['TREATMENT'] == 0)[:, None]])


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 68 and the array at index 1 has size 1